In [1]:
# import data into dataframe

import os
import json
import pandas as pd


with open(os.path.join(os.getcwd(), "data", "pos_neg_copy_y.json"), "r") as file:
    data = json.load(file)

In [2]:
from copyrightDet.preprocess_string import PreprocessString
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import nltk

try:
    from nltk.corpus import stopwords
except Exception:
    nltk.download('stopwords')
    from nltk.corpus import stopwords

stops = list(stopwords.words('english'))

stops

prePro = PreprocessString()
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b",
                             preprocessor=prePro.preprocess,
                             # stop_words=stops
                             )

X = vectorizer.fit_transform(data.keys())
X = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out(), index=data.keys())

X["is_no_copyright_statement"] = data.values()
X.head()


,0,00,000,0002,000aa,00readme,01,02,021,03,...,î448,î454,î458,î482,ð,ó,ö,ø,ɉ,is_no_copyright_statement
Copyrights for code taken from ext2:,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"copyright notice, this list of conditions and the following",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"copyright notice, this list of conditions and the following disclaimer",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"Parts derived from drivers/block/rd.c, and drivers/block/loop.c, copyright",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Original copyright notice follows:,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
print(X.groupby(X["is_no_copyright_statement"]).size())

X.drop_duplicates(inplace=True)
print(X.groupby(X["is_no_copyright_statement"]).size())

is_no_copyright_statement
0    4640
1    5007
dtype: int64
is_no_copyright_statement
0    4353
1    4964
dtype: int64


In [4]:
# find most used words in entire dataset

X_t = X.transpose()

top_dict = {}
for c in X_t.columns:
    top = X_t[c].sort_values(ascending=False).head(30)
    for i in range(len(top.index)):
        if top_dict.get(top.index[i]):
            top_dict[top.index[i]] += top.values[i]
        else:
            top_dict[top.index[i]] = top.values[i]
            

top_df = pd.DataFrame(top_dict.values(), columns=["count"], index=top_dict.keys()).sort_values(by=["count"], ascending=False)
top_df.head(15)

,count
copyright,12500
is_no_copyright_statement,4961
the,4493
of,3050
and,1870
to,1530
in,1315
a,1293
inc,1027
is,970


In [5]:
# find most used words in class 0

X_t = X.transpose()

top_dict = {}
for c in X_t.columns:
    top = X_t[c].sort_values(ascending=False).head(30)
    for i in range(len(top.index)):
        if top_dict.get(top.index[i]):
            top_dict[top.index[i]] += top.values[i]
        else:
            top_dict[top.index[i]] = top.values[i]
            

top_df = pd.DataFrame(top_dict.values(), columns=["count"], index=top_dict.keys()).sort_values(by=["count"], ascending=False)
top_df.head(15)

,count
copyright,12500
is_no_copyright_statement,4961
the,4493
of,3050
and,1870
to,1530
in,1315
a,1293
inc,1027
is,970


In [6]:
import numpy as np

cor_matrix = X.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))

to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.90)]
print(to_drop)

df1 = X.drop(X.columns[to_drop], axis=1)
print(df1.head())